In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
print(torch.__version__) # tutorial은 1.12.0+cu102 base

1.13.1+cu117


# 데이터 작업하기
- PyTorch에는 데이터 작업을 위한 기본 요소 두가지인`torch.utills.data.DataLoader` 와 `torch.utils.data.Dataset` 이 있다.
    
    
- `Dataset`은 샘플과 라벨을 저장하고, `DataLoader`는 `Dataset`을 순회하는 Iterable 객체로 감싼다.

In [4]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

/opt/conda/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
print(training_data, '\n','===='*8)
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() 
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [13]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

- `Dataset`을 `DataLoader`의 인자로 전달한다. 이는 데이터셋을 Iterable 객체로 감싸고 자동화된 배치, 샘플링, 셔플 및 멀티프로세스 데이터 로딩을 지원한다.

In [14]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [21]:
for x,y in test_dataloader:
    print(f'Shape of x [N, C, H, W]: {x.shape}, {x.dtype}')
    print(f'Shape of y:[Label Number]: {y.shape}, {y.dtype}')
    break

Shape of x [N, C, H, W]: torch.Size([64, 1, 28, 28]), torch.float32
Shape of y:[Label Number]: torch.Size([64]), torch.int64


- 데이터는 `(64, 1, 28, 28)` 로 `28 * 28 * 1` 의 이미지를 `64개` 갖고있다.
    - 64개인 이유는 batch_size가 64이기 때문.
- 이미지가 64개이기에 Label 또한 64이다.

## 모델 만들기

- PyTorch에서 신경망 모델은 `nn.Module`을 상속받는 클래스를 생성하여 정의한다. `__init__` 함수에서 신경망의 계층을 정의하고 `forward` 함수에서 신경망에 데이터를 어떻게 전달할지 지정한다. 가능하다면 `GPU`를 사용하여 연산을 가속한다.

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 가능한 디바이스는 {device} 입니다.")

사용 가능한 디바이스는 cuda 입니다.


In [25]:
class NeuralNetwork(nn.Module):
    """
    1. flatten을 통해 28*28*1 의 크기였던 사진을 1차원으로 바꾸어줌. -> 784 size
    2. Sequencial한 linear layer를 통해 784 -> 512 -> 10 의 크기로 변환 해줌
    3. 변환된 값을 return
    """
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),
                                               nn.ReLU(),
                                               nn.Linear(512,512),
                                               nn.ReLU(),
                                               nn.Linear(512,10)
                                              )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 매개변수 최적화하기
- 모델을 학습하려면 `손실 함수(loss function)`과 `옵티마이저(optimizer)`가 필요하다.

In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [54]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        # x = 이미지 , y = 라벨
        
        pred = model(x)
        # pred = 예측 텐서

        loss = loss_fn(pred, y)
        optimizer.zero_grad() # 가중치 초기화
        loss.backward() # 역전파
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [55]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [56]:
# 1. size = len(dataloader.dataset)

print(train_dataloader.dataset, '\n','===='*6)
print('size :', len(train_dataloader.dataset))

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() 
size : 60000


In [57]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.033136 [    0/60000]
loss: 2.017495 [ 6400/60000]
loss: 1.914361 [12800/60000]
loss: 1.950931 [19200/60000]
loss: 1.862954 [25600/60000]
loss: 1.802603 [32000/60000]
loss: 1.826298 [38400/60000]
loss: 1.726144 [44800/60000]
loss: 1.746195 [51200/60000]
loss: 1.651227 [57600/60000]
Test Error: 
 Accuracy: 61.3%, Avg loss: 1.653641 

Epoch 2
-------------------------------
loss: 1.698215 [    0/60000]
loss: 1.669007 [ 6400/60000]
loss: 1.518018 [12800/60000]
loss: 1.589838 [19200/60000]
loss: 1.471955 [25600/60000]
loss: 1.450207 [32000/60000]
loss: 1.467201 [38400/60000]
loss: 1.379751 [44800/60000]
loss: 1.418831 [51200/60000]
loss: 1.318145 [57600/60000]
Test Error: 
 Accuracy: 63.4%, Avg loss: 1.336320 

Epoch 3
-------------------------------
loss: 1.400632 [    0/60000]
loss: 1.385539 [ 6400/60000]
loss: 1.219246 [12800/60000]
loss: 1.320307 [19200/60000]
loss: 1.200355 [25600/60000]
loss: 1.214832 [32000/60000]
loss: 1.234418 [38400/

## 모델 저장하기

- 모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)을 직렬화(serialize)하는 것입니다.

In [58]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


## 모델 불러오기

- 모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됩니다.

In [60]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [74]:
import random

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
for i in range(3):
    random_number = random.choice(list(range(len(test_data))))
    x, y = test_data[random_number][0], test_data[random_number][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'{i}번째 예측값: "{predicted}", 실제값: "{actual}"')

0번째 예측값: "T-shirt/top", 실제값: "T-shirt/top"
1번째 예측값: "T-shirt/top", 실제값: "T-shirt/top"
2번째 예측값: "Pullover", 실제값: "Pullover"
